# PyZX interoperability

We can use [PyZX](https://github.com/Quantomatic/pyzx.git) to load intermediate representations like QASM to simulate with PyQrack, as well as to optimize circuits with the ZX-calculus before simulating them.

In [1]:
import sys; sys.path.insert(0,'..')
import os
from collections import Counter
import pyzx as zx
from pyqrack import QrackSimulator

If the file has the expected extension and other format properties, we should be able to load any intermediate representation that PyZX knows with `pyzx.Circuit.load()`.

In [2]:
fname = os.path.join('qasm','qasmbench_ising26.qasm')
circ = zx.Circuit.load(fname)
circ.gates

[HAD(0),
 HAD(1),
 HAD(2),
 HAD(3),
 HAD(4),
 HAD(5),
 HAD(6),
 HAD(7),
 HAD(8),
 HAD(9),
 HAD(10),
 HAD(11),
 HAD(12),
 HAD(13),
 HAD(14),
 HAD(15),
 HAD(16),
 HAD(17),
 HAD(18),
 HAD(19),
 HAD(20),
 HAD(21),
 HAD(22),
 HAD(23),
 HAD(24),
 HAD(25),
 ZPhase(0,phase=-52191293/93579044),
 ZPhase(1,phase=52191293/93579044),
 ZPhase(1,phase=52191293/93579044),
 CNOT(0,1),
 ZPhase(1,phase=-52191293/93579044),
 CNOT(0,1),
 ZPhase(2,phase=-6839239/19445588),
 ZPhase(3,phase=6839239/19445588),
 ZPhase(3,phase=6839239/19445588),
 CNOT(2,3),
 ZPhase(3,phase=-6839239/19445588),
 CNOT(2,3),
 ZPhase(4,phase=-16222424/70325907),
 ZPhase(5,phase=16222424/70325907),
 ZPhase(5,phase=16222424/70325907),
 CNOT(4,5),
 ZPhase(5,phase=-16222424/70325907),
 CNOT(4,5),
 ZPhase(6,phase=-13236349/29102314),
 ZPhase(7,phase=13236349/29102314),
 ZPhase(7,phase=13236349/29102314),
 CNOT(6,7),
 ZPhase(7,phase=-13236349/29102314),
 CNOT(6,7),
 ZPhase(8,phase=39655134/74371259),
 ZPhase(9,phase=-39655134/74371259),
 

Before optimizing the circuit, we need to convert it to a graph representation.

In [3]:
g = circ.to_graph()
g

Graph(382 vertices, 406 edges)

Then, we can optimize the graph.

In [4]:
zx.simplify.full_reduce(g)
g

Graph(128 vertices, 127 edges)

In [5]:
circ = zx.extract_circuit(g)
circ.gates

[HAD(0),
 CZ(0,1),
 HAD(1),
 CZ(1,2),
 CZ(0,2),
 HAD(2),
 CZ(2,3),
 CZ(1,3),
 CZ(0,3),
 HAD(3),
 CZ(3,4),
 CZ(2,4),
 CZ(1,4),
 CZ(0,4),
 HAD(4),
 CZ(4,5),
 CZ(3,5),
 CZ(2,5),
 CZ(1,5),
 CZ(0,5),
 HAD(5),
 CZ(5,6),
 CZ(4,6),
 CZ(3,6),
 CZ(2,6),
 CZ(1,6),
 CZ(0,6),
 HAD(6),
 CZ(6,7),
 CZ(5,7),
 CZ(4,7),
 CZ(3,7),
 CZ(2,7),
 CZ(1,7),
 CZ(0,7),
 HAD(7),
 CZ(7,8),
 CZ(6,8),
 CZ(5,8),
 CZ(4,8),
 CZ(3,8),
 CZ(2,8),
 CZ(1,8),
 CZ(0,8),
 HAD(8),
 CZ(8,9),
 CZ(7,9),
 CZ(6,9),
 CZ(5,9),
 CZ(4,9),
 CZ(3,9),
 CZ(2,9),
 CZ(1,9),
 CZ(0,9),
 HAD(9),
 CZ(9,10),
 CZ(8,10),
 CZ(7,10),
 CZ(6,10),
 CZ(5,10),
 CZ(4,10),
 CZ(3,10),
 CZ(2,10),
 CZ(1,10),
 CZ(0,10),
 HAD(10),
 CZ(10,11),
 CZ(9,11),
 CZ(8,11),
 CZ(7,11),
 CZ(6,11),
 CZ(5,11),
 CZ(4,11),
 CZ(3,11),
 CZ(2,11),
 CZ(1,11),
 CZ(0,11),
 HAD(11),
 CZ(11,12),
 CZ(10,12),
 CZ(9,12),
 CZ(8,12),
 CZ(7,12),
 CZ(6,12),
 CZ(5,12),
 CZ(4,12),
 CZ(3,12),
 CZ(2,12),
 CZ(1,12),
 CZ(0,12),
 HAD(12),
 CZ(12,13),
 CZ(11,13),
 CZ(10,13),
 CZ(9,13),
 CZ(8,13),
 CZ(7,

Once the graph is loaded and optionally optimized, we can feed it directly to a QrackSimulator for immediate simulation

In [6]:
%%time
sim = QrackSimulator(pyzxCircuit=circ, is1QbFusion=False)
Counter(sim.measure_shots(range(circ.qubits), 256))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
CPU times: user 1min 11s, sys: 775 ms, total: 1min 12s
Wall time: 18.7 s


Counter({20036: 1,
         111689: 1,
         328395: 1,
         946140: 1,
         1286305: 1,
         1429663: 1,
         1435387: 1,
         1555194: 1,
         1788323: 1,
         2006468: 1,
         2178855: 1,
         2245346: 1,
         3173711: 1,
         3377912: 1,
         3777680: 1,
         3951049: 1,
         4192598: 1,
         4357808: 1,
         4789988: 1,
         4836066: 1,
         5083279: 1,
         5144709: 1,
         5348370: 1,
         5423040: 1,
         5779631: 1,
         5821597: 1,
         5833338: 1,
         6071359: 1,
         6124593: 1,
         6223263: 1,
         6533874: 1,
         7355915: 1,
         8222913: 1,
         8334061: 1,
         8383694: 1,
         8507051: 1,
         8528070: 1,
         8882681: 1,
         10103389: 1,
         10327083: 1,
         10419665: 1,
         11106532: 1,
         11509867: 1,
         11616063: 1,
         11647273: 1,
         12048989: 1,
         12398173: 1,
         